In [ ]:
import os
import shutil
from ast import literal_eval
from glob import glob

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load Data

## Part 1: Images

Create a dataframe containing the image names

In [ ]:
# Location of dataset
DATASET_PATH = "../data/01_raw"

# List all images in the folder
image_list = [
    filename.split("/")[-1].split(".")[0]
    for filename in glob(DATASET_PATH + "/images/*.jpg")
]
image_ids = pd.DataFrame(image_list).rename(columns={0: "image_id"})
print("Number of images in folder: {}".format(len(image_ids)))

image_ids.head()

## Part 2: Annotations

Add the bounding box informations to the dataframe. A bounding box is a rectangle around the object detected. 

In [ ]:
# convert a string record into a valid python object
def convert_string_to_python_object(x):
    return literal_eval(x.rstrip("\r\n"))


# read the CSV with annotations
labels = pd.read_csv(
    DATASET_PATH + "/annotations.csv",
    converters={"bounds": convert_string_to_python_object},
)


labels.head()

# Train Test Split

The train-test split procedure is used to estimate the performance of machine learning algorithms when they are used to make predictions on data not used to train the model.

In [ ]:
train_image_names, test_image_names = train_test_split(image_list, test_size=0.2)
val_image_names, test_image_names = train_test_split(test_image_names, test_size=0.5)

print(
    f"The trainings dataset contains {len(train_image_names)} images. Thats {round(len(train_image_names)/len(image_list) *100)}%."
)
print(
    f"The test dataset contains {len(test_image_names)} images. Thats {round(len(test_image_names)/len(image_list) *100)}%."
)
print(
    f"The validation dataset contains {len(val_image_names)} images. Thats {round(len(val_image_names)/len(image_list) *100)}%."
)

In [ ]:
# add bounding box information
train_df = labels[labels["image_id"].isin(train_image_names)]
valid_df = labels[labels["image_id"].isin(val_image_names)]
test_df = labels[labels["image_id"].isin(test_image_names)]

# write datasets to disk
os.makedirs("../data/04_feature/annotations", exist_ok=True)
train_df.to_csv("../data/04_feature/annotations/train.csv", index=False)
valid_df.to_csv("../data/04_feature/annotations/valid.csv", index=False)
test_df.to_csv("../data/04_feature/annotations/test.csv", index=False)

# copy images to model_input folder
os.makedirs("../data/04_feature/images/train", exist_ok=True)
for image_name in train_image_names:
    shutil.copy(
        f"../data/01_raw/images/{image_name}.jpg",
        f"../data/04_feature/images/train/{image_name}.jpg",
    )
os.makedirs("../data/04_feature/images/validation", exist_ok=True)
for image_name in val_image_names:
    shutil.copy(
        f"../data/01_raw/images/{image_name}.jpg",
        f"../data/04_feature/images/validation/{image_name}.jpg",
    )
os.makedirs("../data/04_feature/images/test", exist_ok=True)
for image_name in test_image_names:
    shutil.copy(
        f"../data/01_raw/images/{image_name}.jpg",
        f"../data/04_feature/images/test/{image_name}.jpg",
    )

# Convert data to tensorflow records dataset

The TFRecord format is a simple format for storing a sequence of binary records.

In [ ]:
from __future__ import annotations

from ast import literal_eval
from pathlib import Path

import tensorflow.compat.v1 as tf
from object_detection.utils import dataset_util

## Define util functions

In [ ]:
def create_tf_records_from_files(
    image_path: str | Path, annotations_file_path: str | Path
) -> list[tf.train.Example]:

    image_path = Path(image_path)
    annotations_file_path = Path(annotations_file_path)

    annotations = pd.read_csv(annotations_file_path)

    def expand_bbox_coordinates(row: pd.Series):
        bbox_as_tuple = literal_eval(row["bounds"])
        row["bbox_x1"] = bbox_as_tuple[0]
        row["bbox_y1"] = bbox_as_tuple[1]
        row["bbox_x2"] = bbox_as_tuple[2]
        row["bbox_y2"] = bbox_as_tuple[3]

        return row

    annotations = annotations.apply(expand_bbox_coordinates, 1)

    annotations.drop(["class", "bounds"], axis=1, inplace=True)

    image_filenames = os.listdir(image_path)

    tf_records = []

    for image_filename in image_filenames:
        with tf.gfile.GFile(image_path / image_filename, "rb") as fid:
            encoded_jpg = fid.read()

        image_width = 2560
        image_height = 2560

        filename = image_filename.encode("utf8")
        image_format = b"jpg"

        annotations_filtered = annotations[
            annotations["image_id"] == image_filename[:-4]
        ]

        xmins = annotations_filtered["bbox_x1"].values
        xmaxs = annotations_filtered["bbox_x2"].values
        ymins = annotations_filtered["bbox_y1"].values
        ymaxs = annotations_filtered["bbox_y2"].values

        class_name = "OST".encode("utf8")

        classes_text = [class_name for i in range(len(xmins))]
        classes = [1 for i in range(len(xmins))]

        tf_records.append(
            tf.train.Example(
                features=tf.train.Features(
                    feature={
                        "image/height": dataset_util.int64_feature(image_height),
                        "image/width": dataset_util.int64_feature(image_width),
                        "image/filename": dataset_util.bytes_feature(filename),
                        "image/source_id": dataset_util.bytes_feature(filename),
                        "image/encoded": dataset_util.bytes_feature(encoded_jpg),
                        "image/format": dataset_util.bytes_feature(image_format),
                        "image/object/bbox/xmin": dataset_util.float_list_feature(
                            xmins
                        ),
                        "image/object/bbox/xmax": dataset_util.float_list_feature(
                            xmaxs
                        ),
                        "image/object/bbox/ymin": dataset_util.float_list_feature(
                            ymins
                        ),
                        "image/object/bbox/ymax": dataset_util.float_list_feature(
                            ymaxs
                        ),
                        "image/object/class/text": dataset_util.bytes_list_feature(
                            classes_text
                        ),
                        "image/object/class/label": dataset_util.int64_list_feature(
                            classes
                        ),
                    }
                )
            )
        )

    return tf_records


def save_tf_records_to_file(
    tf_records: list[tf.train.Example], output_path: str | Path
):
    writer = tf.python_io.TFRecordWriter(output_path)

    for record in tf_records:
        writer.write(record.SerializeToString())
    writer.close()


def label_map(file_path: str, objname: str):
    with open(Path(file_path), "a") as the_file:
        the_file.write("item\n")
        the_file.write("{\n")
        the_file.write("id :{}".format(int(1)))
        the_file.write("\n")
        the_file.write("name :'{0}'".format(str(objname)))
        the_file.write("\n")
        the_file.write("}\n")

## Write data

In [ ]:
# create label map with one object: oil-storage-tank (OSD)
label_map("../data/04_feature/annotations/label_map.pbtxt", "OSD")

# create tfrecords from images and annotations 
tf_train_records = create_tf_records_from_files(
    "../data/04_feature/images/train/",
    "../data/04_feature/annotations/train.csv"
)

tf_test_records = create_tf_records_from_files(
    "../data/04_feature/images/test/",
    "../data/04_feature/annotations/test.csv"
)

tf_valid_records = create_tf_records_from_files(
    "../data/04_feature/images/validation/",
    "../data/04_feature/annotations/valid.csv"
)

# write tfrecord files to disk 
save_tf_records_to_file(tf_train_records, "../data/05_model_input/train.record")

save_tf_records_to_file(tf_test_records, "../data/05_model_input/test.record")

save_tf_records_to_file(tf_valid_records, "../data/05_model_input/valid.record")